<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# xDeepFM : the eXtreme Deep Factorization Machine 
This notebook will give you a quick example of how to train an [xDeepFM model](https://arxiv.org/abs/1803.05170). 
xDeepFM \[1\] is a deep learning-based model aims at capturing both lower- and higher-order feature interactions for precise recommender systems. Thus it can learn feature interactions more effectively and manual feature engineering effort can be substantially reduced. To summarize, xDeepFM has the following key properties:
* It contains a component, named CIN, that learns feature interactions in an explicit fashion and in vector-wise level;
* It contains a traditional DNN component that learns feature interactions in an implicit fashion and in bit-wise level.
* The implementation makes this model quite configurable. We can enable different subsets of components by setting hyperparameters like `use_Linear_part`, `use_FM_part`, `use_CIN_part`, and `use_DNN_part`. For example, by enabling only the `use_Linear_part` and `use_FM_part`, we can get a classical FM model.

In this notebook, we test xDeepFM on two datasets: 1) a small synthetic dataset and 2) [Criteo dataset](http://labs.criteo.com/category/dataset)

## 0. Global Settings and Imports

In [2]:
import sys
sys.path.append("../../")
import os
import papermill as pm
from tempfile import TemporaryDirectory

import tensorflow as tf

from reco_utils.recommender.deeprec.deeprec_utils import (
    download_deeprec_resources, prepare_hparams
)
from reco_utils.recommender.deeprec.models.xDeepFM import XDeepFMModel
from reco_utils.recommender.deeprec.IO.iterator import FFMTextIterator

print("System version: {}".format(sys.version))
print("Tensorflow version: {}".format(tf.__version__))

tmpdir = TemporaryDirectory()

System version: 3.6.8 |Anaconda, Inc.| (default, Dec 30 2018, 01:22:34) 
[GCC 7.3.0]
Tensorflow version: 1.12.0


#### Parameters

In [3]:
EPOCHS_FOR_SYNTHETIC_RUN = 15
EPOCHS_FOR_CRITEO_RUN = 30
BATCH_SIZE_SYNTHETIC = 128
BATCH_SIZE_CRITEO = 4096

xDeepFM uses the FFM format as data input: `<label> <field_id>:<feature_id>:<feature_value>`  
Each line represents an instance, `<label>` is a binary value with 1 meaning positive instance and 0 meaning negative instance. 
Features are divided into fields. For example, user's gender is a field, it contains three possible values, i.e. male, female and unknown. Occupation can be another field, which contains many more possible values than the gender field. Both field index and feature index are starting from 1. <br>

## 1. Synthetic data
Now let's start with a small synthetic dataset. In this dataset, there are 10 fields, 1000 fefatures, and label is generated according to the result of a set of preset pair-wise feature interactions. 

In [4]:
data_path = tmpdir.name
yaml_file = os.path.join(data_path, r'xDeepFM.yaml')
train_file = os.path.join(data_path, r'synthetic_part_0')
valid_file = os.path.join(data_path, r'synthetic_part_1')
test_file = os.path.join(data_path, r'synthetic_part_2')
output_file = os.path.join(data_path, r'output.txt')

if not os.path.exists(yaml_file):
    download_deeprec_resources(r'https://recodatasets.blob.core.windows.net/deeprec/', data_path, 'xdeepfmresources.zip')


10.6MB [00:00, 31.0MB/s]                            


#### 1.1 Prepare hyper-parameters
prepare_hparams() will create a full set of hyper-parameters for model training, such as learning rate, feature number, and dropout ratio. We can put those parameters in a yaml file, or pass parameters as the function's parameters (which will overwrite yaml settings).

In [5]:
hparams = prepare_hparams(yaml_file, 
                          FEATURE_COUNT=1000, 
                          FIELD_COUNT=10, 
                          cross_l2=0.0001, 
                          embed_l2=0.0001, 
                          learning_rate=0.001, 
                          epochs=EPOCHS_FOR_SYNTHETIC_RUN,
                          batch_size=BATCH_SIZE_SYNTHETIC)
print(hparams)

[('DNN_FIELD_NUM', None), ('FEATURE_COUNT', 1000), ('FIELD_COUNT', 10), ('MODEL_DIR', None), ('PAIR_NUM', None), ('SUMMARIES_DIR', None), ('activation', ['relu', 'relu']), ('attention_activation', None), ('attention_dropout', 0.0), ('attention_layer_sizes', None), ('batch_size', 128), ('cross_activation', 'identity'), ('cross_l1', 0.0), ('cross_l2', 0.0001), ('cross_layer_sizes', [1]), ('cross_layers', None), ('data_format', 'ffm'), ('dim', 10), ('doc_size', None), ('dropout', [0.0, 0.0]), ('dtype', 32), ('embed_l1', 0.0), ('embed_l2', 0.0001), ('enable_BN', False), ('entityEmb_file', None), ('entity_dim', None), ('entity_embedding_method', None), ('entity_size', None), ('epochs', 15), ('fast_CIN_d', 0), ('filter_sizes', None), ('init_method', 'tnormal'), ('init_value', 0.3), ('is_clip_norm', 0), ('iterator_type', None), ('kg_file', None), ('kg_training_interval', 5), ('layer_l1', 0.0), ('layer_l2', 0.0001), ('layer_sizes', [100, 100]), ('learning_rate', 0.001), ('load_model_name', 'yo

#### 1.2 Create data loader
Designate a data iterator for the model. xDeepFM uses FFMTextIterator. 

In [6]:
input_creator = FFMTextIterator

#### 1.3 Create model
When both hyper-parameters and data iterator are ready, we can create a model:

In [7]:
model = XDeepFMModel(hparams, input_creator, seed=1)

## sometimes we don't want to train a model from scratch
## then we can load a pre-trained model like this: 
#model.load_model(r'your_model_path')

Add CIN part.


/data/anaconda/envs/reco_gpu/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Now let's see what is the model's performance at this point (without starting training):

In [8]:
print(model.run_eval(test_file))

{'auc': 0.5015, 'logloss': 0.7382}


AUC=0.5 is a state of random guess. We can see that before training, the model behaves like random guessing.

#### 1.4 Train model
Next we want to train the model on a training set, and check the performance on a validation dataset. Training the model is as simple as a function call:

In [9]:
model.fit(train_file, valid_file)

at epoch 1 train info: auc:0.524, logloss:0.6969 eval info: auc:0.5005, logloss:0.703
at epoch 1 , train time: 3.3 eval time: 3.3
at epoch 2 train info: auc:0.5462, logloss:0.6897 eval info: auc:0.5044, logloss:0.6973
at epoch 2 , train time: 2.9 eval time: 3.4
at epoch 3 train info: auc:0.5707, logloss:0.6856 eval info: auc:0.5103, logloss:0.6956
at epoch 3 , train time: 2.9 eval time: 3.3
at epoch 4 train info: auc:0.605, logloss:0.6788 eval info: auc:0.5231, logloss:0.6943
at epoch 4 , train time: 3.1 eval time: 3.3
at epoch 5 train info: auc:0.6659, logloss:0.6584 eval info: auc:0.5618, logloss:0.6882
at epoch 5 , train time: 3.0 eval time: 3.3
at epoch 6 train info: auc:0.7705, logloss:0.587 eval info: auc:0.6657, logloss:0.6498
at epoch 6 , train time: 3.0 eval time: 3.3
at epoch 7 train info: auc:0.8616, logloss:0.4726 eval info: auc:0.7745, logloss:0.5711
at epoch 7 , train time: 3.0 eval time: 3.3
at epoch 8 train info: auc:0.9119, logloss:0.3833 eval info: auc:0.8364, logloss

#### 1.5 Evaluate model

Again, let's see what is the model's performance now (after training):

In [10]:
res_syn = model.run_eval(test_file)
print(res_syn)
pm.record("res_syn", res_syn)

{'auc': 0.9666, 'logloss': 0.253}


If we want to get the full prediction scores rather than evaluation metrics, we can do this:

In [11]:
model.predict(test_file, output_file)

## 2. Criteo data 

Now we have successfully launched an experiment on a synthetic dataset. Next let's try something on a real world dataset, which is a small sample from [Criteo dataset](http://labs.criteo.com/category/dataset). Criteo dataset is a well known industry benchmarking dataset for developing CTR prediction models and it's frequently adopted as evaluation dataset by research papers. The original dataset is too large for a lightweight demo, so we sample a small portion from it as a demo dataset.

In [12]:
print('demo with Criteo dataset')
hparams = prepare_hparams(yaml_file, 
                          FEATURE_COUNT=2300000, 
                          FIELD_COUNT=39, 
                          cross_l2=0.01, 
                          embed_l2=0.01, 
                          layer_l2=0.01,
                          learning_rate=0.002, 
                          batch_size=BATCH_SIZE_CRITEO, 
                          epochs=EPOCHS_FOR_CRITEO_RUN, 
                          cross_layer_sizes=[20, 10], 
                          init_value=0.1, 
                          layer_sizes=[20,20],
                          use_Linear_part=True, 
                          use_CIN_part=True, 
                          use_DNN_part=True)

train_file = os.path.join(data_path, r'cretio_tiny_train')
valid_file = os.path.join(data_path, r'cretio_tiny_valid')
test_file = os.path.join(data_path, r'cretio_tiny_test')

demo with Criteo dataset


In [13]:
model = XDeepFMModel(hparams, FFMTextIterator, seed=1)

# check the predictive performance before the model is trained
print(model.run_eval(test_file)) 
model.fit(train_file, valid_file)
# check the predictive performance after the model is trained
res_real = model.run_eval(test_file)
print(res_real)
pm.record("res_real", res_real)

Add linear part.
Add CIN part.
Add DNN part.
{'auc': 0.514, 'logloss': 0.7289}
at epoch 1 train info: auc:0.6559, logloss:0.5407 eval info: auc:0.6604, logloss:0.5365
at epoch 1 , train time: 27.3 eval time: 20.9
at epoch 2 train info: auc:0.7055, logloss:0.5154 eval info: auc:0.7062, logloss:0.514
at epoch 2 , train time: 22.5 eval time: 20.8
at epoch 3 train info: auc:0.7269, logloss:0.5039 eval info: auc:0.7254, logloss:0.5042
at epoch 3 , train time: 22.7 eval time: 22.6
at epoch 4 train info: auc:0.7383, logloss:0.497 eval info: auc:0.7351, logloss:0.4981
at epoch 4 , train time: 24.9 eval time: 23.1
at epoch 5 train info: auc:0.7447, logloss:0.4933 eval info: auc:0.7401, logloss:0.495
at epoch 5 , train time: 24.8 eval time: 23.1
at epoch 6 train info: auc:0.7484, logloss:0.4911 eval info: auc:0.7428, logloss:0.4933
at epoch 6 , train time: 24.6 eval time: 23.2
at epoch 7 train info: auc:0.7508, logloss:0.4896 eval info: auc:0.7444, logloss:0.4922
at epoch 7 , train time: 24.8 ev

## Reference
\[1\] Lian, J., Zhou, X., Zhang, F., Chen, Z., Xie, X., & Sun, G. (2018). xDeepFM: Combining Explicit and Implicit Feature Interactions for Recommender Systems. Proceedings of the 24th ACM SIGKDD International Conference on Knowledge Discovery \& Data Mining, KDD 2018, London, UK, August 19-23, 2018.<br>

In [14]:
# Cleanup
tmpdir.cleanup()